In [13]:
import pandas as pd
import re
df_ielts_hf = pd.read_excel(r'ielts-evaluations-simple-ds_cleaned.xlsx')
df_ielts_hf.head()


,Unnamed: 0,prompt,essay,evaluation,band
0,0,Interviews form the basic criteria for most la...,It is believed by some experts that the tradit...,**Task Achievement: [7]**\nThe essay effective...,7.5\n\n\n\n\n_x000D__x000D__x000D__x000D__x000...
1,1,Interviews form the basic selecting criteria f...,Nowadays numerous huge firms allocate an inter...,**Task Achievement:** 5.0\n- The candidate has...,5.0\n\n\n\n\n_x000D__x000D__x000D__x000D__x000...
2,2,Interview form the basic selection criteria fo...,The interview section is the most vital part o...,## Task Achievement:\n- The candidate has effe...,5.5\n\n\n\n\n_x000D__x000D__x000D__x000D__x000...
3,3,Interviews form the basic selection criteria f...,It is argued that the best method to recruit e...,## Task Achievement:\n- The candidate has adeq...,5.5\n\n\n\n\n_x000D__x000D__x000D__x000D__x000...
4,4,Interviews from the basic selecting criteria f...,Nowadays many companies conduct interviews bef...,**Task Achievement:**\n\nThe essay adequately ...,4\n\n\n\n\n_x000D__x000D__x000D__x000D__x000D_...


In [14]:
df_cleaned_final = df_ielts_hf.copy()


# Define patterns for extracting scores and feedback for each category
categories = {
    "Task Achievement": r"(?:\*\*|##|###)?\s*Task Achievement[:\s*\-]+([\d\.]+)?\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Coherence and Cohesion": r"(?:\*\*|##|###)?\s*Coherence and Cohesion[:\s*\-]+([\d\.]+)?\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Lexical Resource": r"(?:\*\*|##|###)?\s*Lexical Resource[:\s*\-]+([\d\.]+)?\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Grammatical Range and Accuracy": r"(?:\*\*|##|###)?\s*Grammatical Range and Accuracy[:\s*\-]+([\d\.]+)?\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Overall Band Score": r"(?:\*\*|##|###)?\s*Overall Band Score[:\s*\-]+([\d\.]+)?\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Feedback - Strengths": r"(?:\*\*|##|###)?\s*Feedback and Additional Comments - Strengths[:\s*\-]+\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Feedback - Areas for Improvement": r"(?:\*\*|##|###)?\s*Feedback and Additional Comments - Areas for Improvement[:\s*\-]+\s*\n*(.*?)(?=\*\*|##|###|$)",
    "Suggestions for Improvement": r"(?:\*\*|##|###)?\s*Suggestions for Improvement[:\s*\-]+\s*\n*(.*?)(?=\*\*|##|###|$)",
}
# Refined function to extract scores and feedback more safely
def extract_category_data_safe(text, pattern):
    match = re.search(pattern, text, re.DOTALL)
    if match:
        score = match.group(1).strip() if match.lastindex and match.lastindex >= 1 and match.group(1) else None
        feedback = match.group(2).strip() if match.lastindex and match.lastindex >= 2 and match.group(2) else None
        return score, feedback
    return None, None

# Apply extraction for each category safely
for category, pattern in categories.items():
    df_cleaned_final[f"{category} Score"], df_cleaned_final[f"{category} Feedback"] = zip(*df_cleaned_final['evaluation'].astype(str).apply(lambda x: extract_category_data_safe(x, pattern)))

# Select relevant columns for display
columns_to_display = [col for col in df_cleaned_final.columns if "Score" in col or "Feedback" in col]


In [16]:
df_cleaned_final.head(100)

# export the cleaned data to a new excel file
df_cleaned_final.to_excel("ielts-evaluations-cleanedv2.xlsx", index=False)

In [ ]:
# Define functions to extract scores, comments, and feedback sections

def extract_score(text, category):
    """Extracts scores enclosed in brackets [X] or appearing directly after category."""
    pattern = rf"(?:\*\*|##)?\s*{category}:?\s*(?:\[(\d+(\.\d+)?)\]|(\d+(\.\d+)?))"
    match = re.search(pattern, text)
    return float(match.group(1) or match.group(3)) if match else None

def extract_comment(text, category):
    """Extracts comments following the category heading."""
    pattern = rf"(?:\*\*|##)?\s*{category}:?\s*(?:\[(\d+(\.\d+)?)\]|\d+(\.\d+)?)?\s*\n(.*?)(?=\n(?:\*\*|##)|\Z)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(4).strip() if match and match.group(4) else None

def extract_feedback_section(text, section_name):
    """Extracts full feedback sections."""
    pattern = rf"(?:\*\*|##)?\s*{section_name}:?\s*\n(.*?)(?=\n(?:\*\*|##)|\Z)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

def extract_suggestions(text):
    """Extracts suggestions for improvement."""
    pattern = r"(?:\*\*|##)?\s*Suggestions for Improvement:?\s*\n(.*?)(?=\n(?:\*\*|##)|\Z)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

# Apply functions to extract scores and comments into new columns
df_cleaned_final = df_ielts_hf.copy()  # Preserve original data

categories = [
    "Task Achievement", "Coherence and Cohesion", "Lexical Resource", 
    "Grammatical Range and Accuracy", "Overall Band Score"
]

for category in categories:
    df_cleaned_final[f"{category} Score"] = df_cleaned_final["evaluation"].apply(lambda x: extract_score(x, category))
    df_cleaned_final[f"{category} Comment"] = df_cleaned_final["evaluation"].apply(lambda x: extract_comment(x, category))

# Extract feedback sections
df_cleaned_final["Strengths"] = df_cleaned_final["evaluation"].apply(lambda x: extract_feedback_section(x, "Feedback and Additional Comments - Strengths"))
df_cleaned_final["Areas for Improvement"] = df_cleaned_final["evaluation"].apply(lambda x: extract_feedback_section(x, "Feedback and Additional Comments - Areas for Improvement"))
df_cleaned_final["Suggestions for Improvement"] = df_cleaned_final["evaluation"].apply(lambda x: extract_suggestions(x))

# Display the final cleaned dataset
df_cleaned_final.head(100)
